In [10]:
import pandas as pd
import numpy as np
import ujson
from collections import defaultdict

`compact2.json` is generated by running `cat dblp-ref-* | jq -c '[.id, .year, .references, .authors, .title]' > compact2.json`

In [11]:
vals = []
with open("dblp-ref/compact2.json") as f:
    for l in f:
        vals.append(ujson.loads(l))

df = pd.DataFrame(vals, columns=["id","year","references","authors","title"])
df.set_index("id", inplace=True)

`dblp.flow` is generated by running: ```./infomap dblp.pajek output -t --inner-parallelization -i pjk```

In [12]:
vals = []
with open("output/dblp.flow") as f:
    vals = [l.split() for l in f]
    
flows = pd.DataFrame(list(map(lambda x: [float(x[0]), x[1]], vals)), columns=["flow", "id"])
flows.set_index("id", inplace=True) 

In [13]:
counts = defaultdict(int)
for row in df.itertuples():
    if row.references:
        for pid in row.references:
            counts[pid] += 1
in_cites = pd.DataFrame.from_dict(data=counts, orient="index")
in_cites.rename(columns={0:"in_cite"}, inplace=True)

In [17]:
df["out_cite"] = df.apply(lambda r: len(r.references) if r.references else 0, axis=1)
df["author_cnt"] = df.apply(lambda r: len(r.authors) if r.authors else 0, axis=1)
df["title_len"] = df.apply(lambda r: len(r.title) if r.title else 0, axis=1)

In [18]:
dfa = df.join([flows, in_cites])

In [19]:
dfa.to_pickle("data.pickle")